In [9]:

import numpy as np
import torch as pt
import torch.nn as nn
import torch.nn.functional as f
import torch.optim as optim
import collections as coll
from mpl_toolkits import mplot3d
import matplotlib.pyplot as plt
from matplotlib import cm
import gc
import numpy as np
import torch as pt
import torch
import torch.nn as nn
import torch

def print_gpu_memory():
    allocated_memory = torch.cuda.memory_allocated() / 1024 ** 3
    cached_memory = torch.cuda.memory_reserved() / 1024 ** 3
    print(f"Allocated GPU Memory: {allocated_memory:.2f} GB")
    print(f"Cached GPU Memory: {cached_memory:.2f} GB")

print_gpu_memory()
import time as TIME
import torch_tensorrt

from FHNCUDAlib import FHNCUDA
import numpy as np
import torch 

loaded_module = nn.Linear(10,10)
#loaded_module.load_state_dict()

#print(loaded_module.state_dict())
st=torch.load('network.0.pth')
#print(st)


pt.set_grad_enabled (False) 
numinputs=1
numoutputs=2
class Net(nn.Module):
    def __init__(self, numinputs, numoutputs, numlayers=4, H=10):
        super(Net, self).__init__()

        self.layers = nn.ModuleList()
        self.layers.append(nn.utils.weight_norm(nn.Linear(numinputs, H), name='weight', dim=0).cuda())

        for _ in range(numlayers - 1):
            self.layers.append(nn.utils.weight_norm(nn.Linear(H, H), name='weight', dim=0).cuda())

        self.final_layer = nn.Linear(H, numoutputs).cuda()

        for layer in self.layers:
            layer.eval()
        self.final_layer.eval()

    def forward(self, x):
        for layer in self.layers:
            x = f.silu(layer(x))

        return self.final_layer(x)

    def load(self, od):
        for k, v in od.items():
            if k.startswith('_impl.layers'):
                layer_num = int(k.split('.')[2])
                layer = self.layers[layer_num]
                if k.endswith('linear.weight'):
                    layer.weight_v.data = v
                    layer.weight_v.requires_grad = False
                elif k.endswith('linear.weight_g'):
                    layer.weight_g.data = v
                    layer.weight_g.requires_grad = False
                elif k.endswith('linear.bias'):
                    layer.bias.data = v
                    layer.bias.requires_grad = False
            elif k == '_impl.final_layer.linear.weight':
                self.final_layer.weight.data = v
                self.final_layer.weight.requires_grad = False
            elif k == '_impl.final_layer.linear.bias':
                self.final_layer.bias.data = v
                self.final_layer.bias.requires_grad = False

    def __prepare_scriptable__(self):
        for layer in self.layers:
            for hook in layer._forward_pre_hooks.values():
                if hook.__module__ == "torch.nn.utils.weight_norm" and hook.__class__.__name__ == "WeightNorm":
                    torch.nn.utils.remove_weight_norm(layer)
        return self

import itertools



def Modelrun_s(x,M):
   
    gc.collect()
    torch.cuda.empty_cache()
    my2dspace = pt.tensor(x, requires_grad=False).float().cuda()
    M.eval()
    start_time = TIME.time()
    myOutput = M(my2dspace)
    reftime = TIME.time()- start_time
 
    myCPUOutput = myOutput.cpu()


    uu = myCPUOutput.numpy()

    #print('uu: ', uu.T[0])

    myCPUOutput.squeeze().detach().numpy()
    gc.collect()
    torch.cuda.empty_cache()
    return uu,reftime


def Modelrun(x, M, batch_size=10*20*10*200):

    my2dspace = x
    M.eval()
        
    num_samples = my2dspace.shape[0]
  

    uu_list = []
   

    reftime =0
    for i in range(0, num_samples, batch_size):
        gc.collect()
        torch.cuda.empty_cache()
        batch_input =torch.tensor(my2dspace[i:i+batch_size], requires_grad=False).float().cuda() 
        start_time = TIME.time()
        batch_output = M(batch_input)
        reftime =reftime+ TIME.time() - start_time
        uu_list.append(batch_output.cpu().numpy())
        del batch_input
        del batch_output
        print_gpu_memory()
       
 
    uu = np.concatenate(uu_list)
    
    print_gpu_memory()
    return uu, reftime




def compileRT(modelTorch=Net(2,2)):
    gc.collect()
    torch.cuda.empty_cache()
    X=np.zeros((2,5))
    model = torch.jit.script(modelTorch).eval()  # torch module needs to be in eval (not training) mode



    trt_ts_module = torch_tensorrt.compile(modelTorch, inputs = [torch_tensorrt.Input(
            min_shape=(1,4),
            opt_shape=(5000,4),
            max_shape=(20000*26,4),
            dtype=torch.float32)],
        enabled_precisions = torch.float32, # Run with FP32
        workspace_size = 1 << 33
    )


    gc.collect()
    torch.cuda.empty_cache()
    return trt_ts_module




print("Compiling")
net =Net(4,2,10,10)

PATH=""
od=pt.load('network.0.pth')
net.load(od)
net=net.cuda()


model =  compileRT(net)
print("Done compiling")



Allocated GPU Memory: 0.00 GB
Cached GPU Memory: 0.00 GB
Compiling
Done compiling


In [3]:
!nvcc cuda.cu -o a.out  ##Estava seguindo o tutorioal do torch rt tentando encontrar uma forma de converter, tentei remover os pesos para então converte 

In [11]:

gc.collect()
torch.cuda.empty_cache()


print_gpu_memory()
from itertools import product
gc.collect()
torch.cuda.empty_cache()
krange = [(0.08 + 0.03 * i * 0.1) for i in range(0, 100)]
vrange = [(0.004 + 0.1 * i * 0.12) for i in range(0, 20)]
urange = [(0.005 + 0.1 * i * 1) for i in range(0, 100)]


param_list = np.array(list(product(urange,vrange, krange )))




x0=np.array(param_list)
dt,tt=0.0001,5

rate=4000


##Cuda ref
start_time = TIME.time()
u,v ,t,p=FHNCUDA.run(x0,tt,dt,rate)
reftime = TIME.time()- start_time
p=[i/1000 for i in p[0]]
u_ref=np.array(u).flatten()




print("-------------------------CUDA ----------------------------")

###Cuda run
start_time = TIME.time()
u,v ,t,p=FHNCUDA.run(x0,tt,dt*100,rate/100)
cudatime = TIME.time()- start_time
u_num=np.array(u).flatten()
#print(np.unique(t))
p=[i/1000 for i in p[0]]
print("Shape cudapred ",np.shape(u_num))
print("cuda time",p)
print("Error Calculation")
e=((u_ref-u_num)**2)**(1/2)
print("mean",np.mean(e))
m=np.max(e)
print("max",m)

i=[a for a in range(len(e)) if e[a]==m]
x0 = [item for sublist in x0 for item in sublist]
t = [item for sublist in t for item in sublist]




##PRED NETWORK TORCH
print("-------------------------TORCH ----------------------------")
print(np.shape(t))
start_time = TIME.time()
u,net_time=Modelrun(x0,t[1:],net)

u_net=u.T[0].flatten()

print("Shape netpred ",np.shape(u_net))
print("net time", net_time)
print("Error Calculation")
e=((u_ref-u_net)**2)**(1/2)
print("mean",np.mean(e))
m=np.max(e)
print("max",m)
i=[a for a in range(len(e)) if e[a]==m]


plt.plot(u_ref,"b")
plt.plot(u_net,"r")

plt.show()
("-----------------------------------------------------")
#PRED NETWORK JIT
print("-------------------------JIT ----------------------------")

print(np.shape(t))
start_time = TIME.time()
u,net_time=Modelrun(x0,t[1:],torch.jit.script(net).cuda())
u_net=u.T[0].flatten()
print("Shape netpred ",np.shape(u_net))
print("net time", net_time)
print("Error Calculation")
e=((u_ref-u_net)**2)**(1/2)
print("mean",np.mean(e))
m=np.max(e)
print("max",m)
i=[a for a in range(len(e)) if e[a]==m]


plt.plot(u_ref,"b")
plt.plot(u_net,"r")
plt.show()
print("-----------------------------------------------------")

##PRED NETWORK RT
print("-------------------------TENSOR RT ----------------------------")
print(np.shape(t))
start_time = TIME.time()
u,net_time=Modelrun(x0,t[1:],model)
u_net=u.T[0].flatten()
print("Shape netpred ",np.shape(u_net))
print("net time", net_time)
print("Error Calculation")
e=((u_ref-u_net)**2)**(1/2)
print("mean",np.mean(e))
m=np.max(e)
print("max",m)
i=[a for a in range(len(e)) if e[a]==m]


plt.plot(u_ref,"b")
plt.plot(u_net,"r")
plt.show()
print("-----------------------------------------------------")


gc.collect()
torch.cuda.empty_cache()

Allocated GPU Memory: 0.00 GB
Cached GPU Memory: 0.00 GB
6
Number of rows in the CSV file:  200000 13
-------------------------CUDA ----------------------------
6
Number of rows in the CSV file:  200000 13
Shape cudapred  (2600000,)
cuda time [0.000562304, 0.035071327, 0.467486145]
Error Calculation
mean nan
max nan
-------------------------TORCH ----------------------------
(13,)


AttributeError: 'list' object has no attribute 'eval'

In [5]:
!nvcc cuda.cu -o a.out